In [ ]:
import os
import pickle
import json

In [ ]:
os.chdir('/content/')

# clone refnews data repository
!git clone https://github.com/sfschouten/refnews.git

Cloning into 'refnews'...
remote: Enumerating objects: 34, done.
remote: Total 34 (delta 0), reused 0 (delta 0), pack-reused 34
Unpacking objects: 100% (34/34), done.


In [ ]:
os.chdir('/content/')

# clone mwep repository (which containst class files necessary to unpickle)
!git clone https://github.com/sfschouten/multilingual-wiki-event-pipeline.git mwep -b develop

Cloning into 'mwep'...
remote: Enumerating objects: 774, done.
remote: Counting objects: 100% (53/53), done.
remote: Compressing objects: 100% (34/34), done.
remote: Total 774 (delta 22), reused 44 (delta 19), pack-reused 721
Receiving objects: 100% (774/774), 84.18 MiB | 18.64 MiB/s, done.
Resolving deltas: 100% (504/504), done.


In [ ]:
# dependency
!pip install rdflib news_please

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 500 kB 4.2 MB/s 
     |████████████████████████████████| 89 kB 7.5 MB/s 
     |████████████████████████████████| 41 kB 573 kB/s 
     |████████████████████████████████| 270 kB 56.6 MB/s 
     |████████████████████████████████| 40 kB 4.4 MB/s 
     |████████████████████████████████| 43 kB 2.4 MB/s 
     |████████████████████████████████| 54 kB 2.2 MB/s 
     |████████████████████████████████| 263 kB 48.5 MB/s 
     |████████████████████████████████| 3.9 MB 48.2 MB/s 
     |████████████████████████████████| 3.0 MB 38.5 MB/s 
     |████████████████████████████████| 211 kB 56.7 MB/s 
     |████████████████████████████████| 384 kB 51.1 MB/s 
     |████████████████████████████████| 981 kB 41.5 MB/s 
     |████████████████████████████████| 548 kB 53.9 MB/s 
     |████████████████████████████████| 79 kB 8.4 MB/s 
     |████████████████████████████████| 636 

In [ ]:
TOPIC = {
    'Q1241356_en,pilot.bin':  ('human interest / record and achievement'),
    'Q149086_en,pilot.bin':   ('crime, law and justice / crime / homicide'),
    'Q167170_en,pilot.bin':   ('sport / sport event'),
    'Q18515440_en,pilot.bin': ('sport / sports transaction'),
    'Q27318_en,pilot.bin':    ('educaction / educational testing and examinations'),
    'Q350604_en,pilot.bin':   ('conflict, war and peace / armed conflict'),
    'Q43109_en,pilot.bin':    ('politics / election / referenda'),
    'Q45382_en,pilot.bin':    ('conflict, war and peace / coup d\'etat'),
    'Q669262_en,pilot.bin':   ('politics / election / primary election'),
    'Q7590_en,pilot.bin':     ('economy, business and finance / strategy and marketing / transport'),
    'Q8065_en,pilot.bin':     ('disaster, accident and emergency incident / disaster / natural disaster'),
    'Q11822042_en,pilot.bin': ('accident and emergency incident / transportation accident and incident')
}

DATA_DIR = '/content/refnews/data'

collections_by_file = {}
total_articles = 0
total_incidents = 0

# switch to code directory so class files can be found to unpickle
os.chdir(os.path.join('/content/mwep'))

print('Total number of articles and incidents for which we have URIs.')
print("file                     |     #articles / #incidents    |  topic")
print('------------------------------------------------------------------------------------------------------------------------------')

for f in os.listdir(DATA_DIR):
    if not f.endswith(".bin"):
        continue
    
    with open(os.path.join(DATA_DIR, f), 'rb') as pickle_file:
        collection = pickle.load(pickle_file)
        collections_by_file[f] = collection

    nr_incidents = len(collection.incidents)
    nr_articles = sum(len(incident.reference_texts) for incident in collection.incidents)

    total_articles += nr_articles
    total_incidents += nr_incidents

    short = f[f.find('Q'):]
    topic = TOPIC[short]

    print(F"{short:<23}  |  {nr_articles:<6}  /  {nr_incidents:<5}  =  {nr_articles/nr_incidents:>6.2f}  |  {topic}")

print('------------------------------------------------------------------------------------------------------------------------------')
print(F'total                    |  {total_articles:<7} / {total_incidents:<5}')

Total number of articles and incidents for which we have URIs.
file                     |     #articles / #incidents    |  topic
------------------------------------------------------------------------------------------------------------------------------
Q669262_en,pilot.bin     |  2612    /  193    =   13.53  |  politics / election / primary election
Q8065_en,pilot.bin       |  10900   /  1103   =    9.88  |  disaster, accident and emergency incident / disaster / natural disaster
Q11822042_en,pilot.bin   |  16551   /  1822   =    9.08  |  accident and emergency incident / transportation accident and incident
Q45382_en,pilot.bin      |  3416    /  339    =   10.08  |  conflict, war and peace / coup d'etat
Q43109_en,pilot.bin      |  5627    /  722    =    7.79  |  politics / election / referenda
Q350604_en,pilot.bin     |  4155    /  137    =   30.33  |  conflict, war and peace / armed conflict
Q167170_en,pilot.bin     |  7188    /  518    =   13.88  |  sport / sport event
Q18515440_e

In [ ]:
mwep_settings = json.load(open('/content/mwep/config/mwep_settings.json'))

# settings for crawling Wikipedia sources
excluded_domains = set(mwep_settings['newsplease']['excluded_domains'])
title_required = mwep_settings['newsplease']['title_required']
range_start, range_end = mwep_settings['newsplease']['num_chars_range']
num_chars_range = range(int(range_start), int(range_end))
startswith = mwep_settings['newsplease']['startswith']
timeout = mwep_settings['newsplease']['timeout']
illegal_substrings = mwep_settings['newsplease']['illegal_substrings']
illegal_chars_in_title = mwep_settings['newsplease']['illegal_chars_in_title']

In [ ]:
import crawl_utils
import hashlib


for f, collection in collections_by_file.items():
    
    for incident_obj in collection.incidents:
        
        map = {
            ref_text_obj.web_archive_uri: ref_text_obj
            for ref_text_obj in incident_obj.reference_texts
            if 'Wikipedia source' in ref_text_obj.found_by
        }
       
        primary_url_to_ref_text_obj = crawl_utils.get_ref_text_obj_of_primary_reference_texts(
            list(map.keys()),
            timeout,
            startswith=startswith,
            excluded_domains=excluded_domains,
            title_required=True,
            num_chars_range=num_chars_range,
            illegal_substrings=illegal_substrings,
            illegal_chars_in_title=illegal_chars_in_title,
        )

        for wa_url, new_text_obj in primary_url_to_ref_text_obj.items():
            old_text_obj = map[wa_url]

            old_md5_hash = old_text_obj.content
            new_md5_hash = hashlib.md5(new_text_obj.content.encode('utf-8')).hexdigest()

            # replace md5 with content
            old_text_obj.content = new_text_obj.content

            changed = new_md5_hash != old_md5_hash
            if changed:
                print(f'WARNING: text for {old_text_obj.web_archive_uri} changed')

http://web.archive.org/web/201: : 1it [00:00,  1.67it/s]
http://web.archive.org/web/202: : 4it [00:12,  3.17s/it]
http://web.archive.org/web/201: : 2it [00:01,  1.42it/s]
0it [00:00, ?it/s]
http://web.archive.org/web/202: : 8it [00:06,  1.16it/s]
http://web.archive.org/web/202: : 4it [00:17,  4.29s/it]
http://web.archive.org/web/202: : 1it [00:00,  1.76it/s]
https://web.archive.org/web/20: : 7it [01:25, 16.86s/it]

ERROR:newsplease.crawler.simple_crawler:connection/timeout error: https://web.archive.org/web/20120402195749/http://www.indystar.com/apps/pbcs.dll/article?AID=/20080513/NEWS0502/805130391 HTTPSConnectionPool(host='web.archive.org', port=443): Read timed out. (read timeout=8)


https://web.archive.org/web/20: : 9it [01:36, 10.69s/it]
http://web.archive.org/web/202: : 11it [00:18,  1.70s/it]
http://web.archive.org/web/201: : 3it [00:53, 17.72s/it]
http://web.archive.org/web/202: : 77it [02:12,  1.42s/it]

ERROR:newsplease.crawler.simple_crawler:connection/timeout error: http://web.archive.org/web/20220101151849/https://www.cnn.com/2016/06/09/politics/president-barack-obama-endorses-hillary-clinton-in-video/index.html HTTPConnectionPool(host='web.archive.org', port=80): Read timed out. (read timeout=8)


http://web.archive.org/web/202: : 110it [03:16,  1.32s/it]

...

/usr/local/lib/python3.7/dist-packages/dateutil/parser/_parser.py:1212: UnknownTimezoneWarning: tzname EST identified but not understood.  Pass `tzinfos` argument in order to correctly return a timezone-aware datetime.  In a future version, this will raise an exception.
  category=UnknownTimezoneWarning)


https://web.archive.org/web/20: : 13it [00:26,  2.05s/it]
0it [00:00, ?it/s]
http://web.archive.org/web/202: : 22it [00:51,  1.55s/it]

http://web.archive.org/web/202: : 29it [01:06,  2.28s/it]
http://web.archive.org/web/202: : 31it [01:32,  3.00s/it]
http://web.archive.org/web/202: : 1it [00:00,  1.71it/s]
http://web.archive.org/web/202: : 1it [00:01,  1.19s/it]
https://web.archive.org/web/20: : 2it [00:01,  1.21it/s]
http://web.archive.org/web/201: : 2it [00:03,  1.65s/it]
http://web.archive.org/web/202: : 1it [00:01,  1.98s/it]
https://web.archive.org/web/20: : 1it [00:06,  6.28s/it]
http://web.archive.org/web/202: : 3it [00:10,  3.42s/it]
0it [00:00, ?it/s]
http://web.archive.org/web/202: : 5it [00:08,  1.63s/it]
http://web.archive.org/web/202: : 1it [00:06,  6.09s/it]
https://web.archive.org/web/20: : 1it [00:04,  4.36s/it]

Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/newspaper/images.py", line 118, in fetch_url
    p.feed(new_data)
  File "/usr/local/lib/python3.7/dist-packages/PIL/ImageFile.py", line 411, in feed
    im = Image.open(fp)
  File "/usr/local/lib/python3.7/dist-packages/PIL/Image.py", line 2881, in open
    im = _open_core(fp, filename, prefix)
  File "/usr/local/lib/python3.7/dist-packages/PIL/Image.py", line 2867, in _open_core
    im = factory(fp, filename)
  File "/usr/local/lib/python3.7/dist-packages/PIL/ImageFile.py", line 107, in __init__
    self._open()
  File "/usr/local/lib/python3.7/dist-packages/PIL/IcoImagePlugin.py", line 279, in _open
    self.load()
  File "/usr/local/lib/python3.7/dist-packages/PIL/IcoImagePlugin.py", line 295, in load
    im = self.ico.getimage(self.size)
  File "/usr/local/lib/python3.7/dist-packages/PIL/IcoImagePlugin.py", line 161, in getimage
    return self.frame(self.getentryindex(size, bpp))
  File "/usr/local/

https://web.archive.org/web/20: : 17it [02:48,  9.93s/it]
http://web.archive.org/web/202: : 1it [00:01,  1.81s/it]
http://web.archive.org/web/202: : 1it [00:00,  1.69it/s]
http://web.archive.org/web/202: : 4it [01:00, 17.26s/it]